In [22]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [23]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

In [24]:
ratings.rename(columns={'rating':'count'}, inplace=True)

In [25]:
my_favorite = [1193, 661, 914 ,3408 ,2355]

my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'count':[3]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  
    ratings = ratings.append(my_playlist)                            

ratings.tail(10)       

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,1193,3,NaN
1,6041,661,3,NaN
2,6041,914,3,NaN
3,6041,3408,3,NaN
4,6041,2355,3,NaN


In [26]:
from scipy.sparse import csr_matrix

user = ratings['user_id'].nunique()
movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
csr_data_transpose = csr_data.T
csr_data_transpose

als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 훈련한 데이터로 예측한 나의 선호도(내가 선호한 영화/ 그냥 아무영화)

In [31]:
dw, movie2 = ratings.user_id[6041], ratings.movie_id[661]
dw_vector, movie2_vector = als_model.user_factors[dw], als_model.item_factors[661]

np.dot(dw_vector, movie2_vector)

0.24298467

In [32]:
movie0 = ratings.movie_id[500]
movie0_vector = als_model.item_factors[movie0]
np.dot(dw_vector, movie0_vector )

0.108326204

## 비슷한 영화찾기

In [35]:
def get_similar_movie(movie_id):
    movie_id = ratings['movie_id']
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [movie_id[i[0]] for i in similar_movie]
    return similar_movie

get_similar_movie(611)

ValueError: shapes (3953,100) and (836483,100) not aligned: 100 (dim 1) != 836483 (dim 0)

## 추천영화

In [37]:
user = ratings.user_id[6041]

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[ratings.movie_id[i[0]] for i in movie_recommended]

my_movie = ratings.movie_id[611]
explain = als_model.explain(user, csr_data, itemid=my_movie)

KeyError: 3114